## **Imports and Setup**

We begin by importing necessary libraries including Qiskit for quantum computing and NumPy for matrix operations.

In [248]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator
from qiskit_aer import AerSimulator
from numpy import random

## **Defining the Quantum Operators**

Here, we create a 3 x 3 square of tensored Pauli operators that will be measured on the qubits of Alice and Bob.

\begin{matrix}
X \otimes X & Y \otimes Z & Z \otimes Y \\
Y \otimes Y & Z \otimes X & X \otimes Z \\
Z \otimes Z & X \otimes Y & Y \otimes X \\
\end{matrix}


In [249]:
X= Operator.from_label("X")
Y= Operator.from_label("Y")
Z= Operator.from_label("Z")
Magic_Square= [[X^X, Y^Z, Z^Y], [Y^Y, Z^X, X^Z], [Z^Z, X^Y, Y^X]]

## **Finding the Common Eigenbasis**

Now, we will create a function which will return the common change-of-basis matrix for $3$ matrices (which is also the inverse of the matrix formed by stacking the common eigenvectors together) along with the list of eigenvalues for the matrices, grouped together by their eigenvectors.  
Since, the overall process is a little involved, we demonstrate the process by finding a common eigenbasis for the commuting Pauli operators $Y \otimes Y$, $Z \otimes X$ and $X \otimes Z$:

##### **Step 1: Writing the $YY$ operator matrix**

Recall the Pauli $Y$ matrix:  
$$
Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
$$  
The tensor product $Y \otimes Y$ is:  
$$
Y \otimes Y = \begin{pmatrix}
0 & 0 & 0 & -1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
-1 & 0 & 0 & 0 \\
\end{pmatrix}
$$

##### **Step 2: Finding the eigenvectors and eigenvalues of $Y \otimes Y$**

Diagonalizing $Y \otimes Y$, we get eigenvalues:  
$$
\lambda = \{+1, +1, -1, -1\}
$$  
and corresponding eigenvectors form columns of the matrix $U_{\text{initial}}^{-1}$, the eigenbasis of $Y \otimes Y$:  
$$
U_{\text{initial}}^{-1} = \frac{1}{\sqrt{2}} \begin{pmatrix}
0 & 1 & 0 & 1 \\
1 & 0 & 1 & 0 \\
1 & 0 & -1 & 0 \\
0 & -1 & 0 & 1 \\
\end{pmatrix}
$$

##### **Step 3: Express the $Z \otimes X$ operator in the $Y \otimes Y$ eigenbasis**

The $Z \otimes X$ operator is:  
$$
Z \otimes X = \begin{pmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & -1 \\
0 & 0 & -1 & 0 \\
\end{pmatrix}
$$  
We change basis to $Y \otimes Y$ eigenbasis by computing:  
$$
B' = U_{\text{initial}}^{-1} \, (Z \otimes X) \, U_{\text{initial}}
$$  
which yields:  
$$
B' = \begin{pmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

##### **Step 4: Identifying degenerate eigenspaces of $Y \otimes Y$**

Since $YY$ has degenerate eigenvalues, we group indices accordingly:  
- $\lambda=+1$ corresponds to indices $\{0,1\}$  
- $\lambda=-1$ corresponds to indices $\{2,3\}$  

We split $B'$ into blocks based on these groups:  
$$
B' = \begin{pmatrix}
\boxed{\begin{matrix}0 & 1 \\ 1 & 0 \end{matrix}} & 0 \\
0 & \boxed{\begin{matrix}0 & 1 \\ 1 & 0 \end{matrix}} \\
\end{pmatrix}
$$

##### **Step 5: Diagonalizing each degenerate block of $B'$**

Diagonalizing block 1:  
$$
\begin{pmatrix}
0 & 1 \\
1 & 0 \\
\end{pmatrix}
$$  
gives eigenvalues $\pm 1$ with eigenvectors:  
$$
v_1 = \frac{1}{\sqrt{2}} \begin{pmatrix}1 \\ -1 \end{pmatrix}, \quad
v_2 = \frac{1}{\sqrt{2}} \begin{pmatrix}1 \\ 1 \end{pmatrix}
$$  
Since the block 2 is equal to block 1 in this case, we have:
$$
v'_1 = \frac{1}{\sqrt{2}} \begin{pmatrix}1 \\ -1 \end{pmatrix}, \quad
v'_2 = \frac{1}{\sqrt{2}} \begin{pmatrix}1 \\ 1 \end{pmatrix}
$$

##### **Step 6: Constructing the common eigenbasis $U_{\text{final}}$**

To find the common eigenvectors of $Y \otimes Y$ and $Z \otimes X$ (and, also $X \otimes X$) we start with the eigenvectors of $Y \otimes Y$, given by the columns of $U_{\text{initial}}$, grouped by their eigenvalues. Then we apply the block-wise change of basis by multiplying these grouped eigenvectors by the eigenvector matrices of the corresponding diagonal blocks of the transformed $Z \otimes X$ operator.

Explicitly, we write:

$$
U_{\text{initial}}^{-1} = \left[
\begin{array}{c|c}
G & G'
\end{array}
\right],
$$

where,

$$
G = \frac{1}{\sqrt{2}} \begin{pmatrix}
0 & 1 \\
1 & 0 \\
1 & 0 \\
0 & -1 \\
\end{pmatrix}, \quad
G' = \frac{1}{\sqrt{2}} \begin{pmatrix}
0 & 1 \\
-1 & 0 \\
-1 & 0 \\
0 & 1 \\
\end{pmatrix}.
$$

The final matrix of common eigenvectors is then given by:

$$
U_{\text{final}}^{-1} = \left[
\begin{array}{c|c}
G V & G' V'
\end{array}
\right]
$$

where,
$$
V= \left[
\begin{array}{c|c}
v_1&v_2
\end{array}
\right]
,\quad
V'= \left[
\begin{array}{c|c}
v'_1&v'_2
\end{array}
\right].
$$
Thus,
$$
U_{\text{final}}^{-1}= \frac{1}{2} \begin{pmatrix}
1 & -1 & 1 & -1\\
1 & 1 & -1 & -1\\
1 & 1 & -1 & -1\\
-1 & 1 & 1 & -1\\
\end{pmatrix}.
$$
Now, since the columns of this matrix form the common eigenvectors of $Y \otimes Y$, $Z \otimes X$ and $X \otimes Z$, computing the list of eigenvalues (grouped by the eigenvectors) along with the change of basis matrix for the common eigenbasis is quite straightforward and relatively simple.

In [275]:
def change_of_basis_matrix(A, B, C, tolerance=1e-7):

    eigenvalues_A, U_initial= np.linalg.eig(A)  
    U_initial_inverse= np.linalg.inv(U_initial)
    B_prime= U_initial_inverse@ B@ U_initial    

    eigen_groups= []
    taken= set()

    for i in range(4):
        if i in taken:  
            continue
        eigen_group= [i]
        for j in range(i+1, 4):
            if abs(eigenvalues_A[i]- eigenvalues_A[j])< tolerance:
                eigen_group.append(j)
        for k in eigen_group:
            taken.add(k)
        eigen_groups.append(eigen_group)

    common_eigenvectors= []
    
    for g in eigen_groups:
        Block_1= B_prime[np.ix_(g, g)]
        Block_2= np.linalg.eig(Block_1)[1]
        stacked_group_eigenvectors= U_initial[:, g] @ Block_2
        common_eigenvectors.append(stacked_group_eigenvectors)

    U_final_inverse = np.hstack(common_eigenvectors)
    U_final= np.linalg.inv(U_final_inverse)

    D_1= U_final@ A@ U_final_inverse
    D_2= U_final@ B@ U_final_inverse
    D_3= U_final@ C@ U_final_inverse

    eigenvalues_list = [[0, 0, 0] for k in range(4)]
    
    for i in range(4):
        eigenvalues_list[i][0]= D_1[i][i]
        eigenvalues_list[i][1]= D_2[i][i]
        eigenvalues_list[i][2]= D_3[i][i]

    return U_final, eigenvalues_list

## **Defining the Game Logic**

Now, we will create another function that gives in some random input and checks if the output (provided using the quantum strategy) is valid according to the rules of the Mermin- Peres Square Game.

The function would return $1$ if the outputs are valid and $0$ otherwise.


In [276]:
def MPMS_Game(quantum_strategy):

    x, y = random.randint(0, 2), random.randint(0, 2)

    x_list, y_list= [], []

    [x_list, y_list]= quantum_strategy(x, y)

    if (sum(x_list)%2 == 0) and (sum(y_list)%2 == 1) and (x_list[y]== y_list[x]):
        return 1
    return 0

## **Building the Quantum Circuit**

This function constructs the quantum circuit for a single round of the MPMS game.

Initially, Alice and Bob share an entangled quantum state:
$$
\left| \psi \right\rangle = \frac{1}{4} \left( \left| 0101 \right\rangle + \left| 1010 \right\rangle - \left| 1001 \right\rangle - \left| 0110 \right\rangle \right)
$$
where, Alice holds the first and third qubits, while Bob holds the second and fourth. These pairs are entangled using a series of gates (Hadamard, CNOT, and Pauli-Z) to reproduce the above state.

Input-dependent unitaries $A$ and $B$ are then applied to Alice’s and Bob’s qubits respectively. These are the change-of-basis matrices for the corresponding row and column (whose indices have randomly been generated by the ```quantum_strategy``` function given above)

Finally, the qubits are measured, and the classical outcomes are stored. For visualization, the circuit diagram is displayed only once.


In [277]:
circuit_displayed= False

def MPMS_circuit(A, B):

    Alice= QuantumRegister(2, "Alice")
    Bob= QuantumRegister(2, "Bob")
    Result_Alice= ClassicalRegister(2, "Result_Alice")
    Result_Bob= ClassicalRegister(2, "Result_Bob")

    circuit= QuantumCircuit(Alice, Bob, Result_Alice, Result_Bob)

    circuit.x(Alice[0])
    circuit.h(Bob[0])
    circuit.cx(Bob[0], Alice[0])
    circuit.z(Alice[0])
    circuit.barrier()

    circuit.x(Alice[1])
    circuit.h(Bob[1])
    circuit.cx(Bob[1], Alice[1])
    circuit.z(Alice[1])
    circuit.barrier()

    circuit.unitary(A, Alice, "A")
    circuit.unitary(B, Bob, "B")
    circuit.barrier()

    circuit.measure(Alice, Result_Alice)
    circuit.barrier()
    circuit.measure(Bob, Result_Bob)
    
    global circuit_displayed
    if not circuit_displayed:
        display(circuit.draw("mpl"))
        circuit_displayed= True

    return circuit


## **Running the Quantum Circuit**

This function implements the quantum strategy for the MPMS game by taking the measurement choices $x$ and $y$ of the two players as inputs. It begins by computing the change-of-basis matrices ($U_1$ and $U_2$) and their associated eigenbases ($E_1$ and $E_2$) using data from the Magic Square. These matrices define the quantum measurements corresponding to the players' choices.

Next, the function constructs and runs the quantum circuit on a simulator with a single shot. The circuit applies the operators derived from the change-of-basis matrices, and the measurement outcomes are collected as bitstrings.

The measured bits are decoded into integer indices $a$ and $b$, which select specific eigenvectors from the eigenbases. Each eigenvector component originally takes values of either +1 or -1. To convert these quantum measurement results into classical bits, the function maps $-1$ to $1$ and $+1$ to $0$. This ensures that the quantum measurement outcomes are represented as classical binary values compatible with the game's logic.

Finally, the function returns two lists of classical bits corresponding to the measurement results for the players, completing the quantum strategy for the MPMS protocol.


In [278]:
def quantum_strategy(x, y):
    
    U_1, E_1= change_of_basis_matrix(Magic_Square[x][0].data, Magic_Square[x][1].data, Magic_Square[x][2].data)
    U_2, E_2= change_of_basis_matrix(Magic_Square[0][y].data, Magic_Square[1][y].data, Magic_Square[2][y].data) 

    result= AerSimulator().run(MPMS_circuit(Operator(U_1), Operator(U_2)), shots= 1).result()
    statistics= result.get_counts()
    bits= list(statistics.keys())[0].replace(" ","")

    a_0, a_1= int(bits[3]), int(bits[2])
    b_0, b_1= int(bits[1]), int(bits[0])

    a= a_0+ 2*a_1
    b= b_0+ 2*b_1

    return [[int(np.real(1-k)//2) for k in E_1[a]], [int(np.real(1-k)//2) for k in E_2[b]]]

## **Simulating Multiple Rounds of the MPMS Game**

In this section, we simulate the MPMS game $1000$ times using the defined quantum strategy. The total score is accumulated over all games, and the final output shows the fraction of games where the quantum strategy succeeded.


In [284]:
NUM_GAMES = 1000
TOTAL_SCORE = 0

for i in range(NUM_GAMES):
    TOTAL_SCORE += MPMS_Game(quantum_strategy)

print("Fraction of games won:", TOTAL_SCORE / NUM_GAMES)

Fraction of games won: 1.0


Our quantum strategy clearly outperforms any classical strategy (which can win the Mermin- Peres Square Game at most $8$ out of $9$ times, i.e. around $88.9\%$ of the times) by achieveing a perfect win rate every time. This advantage comes from entanglement and contextuality, two powerful features exclusive to quantum logic only.